In [1]:
import os
import keras
import pandas as pd

from keras import preprocessing
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

Load dataset

In [2]:
base_data_path =  '../data/BreaKHis_v1/histology_slides/breast'
magnifications = ['40X', '100X', '200X', '400X']
classes = ['benign', 'malignant']
sub_classes = {
    'benign': ['adenosis', 'fibroadenoma', 'phyllodes_tumor', 'tubular_adenoma'],
    'malignant': ['ductal_carcinoma', 'lobular_carcinoma', 'mucinous_carcinoma', 'papillary_carcinoma']
}

In [3]:
dataset = pd.DataFrame()
for clazz in classes:
    for sub_clazz in sub_classes[clazz]:
        path = os.path.join(base_data_path, clazz, "SOB", sub_clazz)
        for id in os.listdir(path):
            for magnification in magnifications:
                path_to_files = os.path.join(path, id, magnification)
                for file_name in os.listdir(path_to_files):
                    dataset = dataset.append({
                        'id': id,
                        'path': os.path.join(path_to_files, file_name),
                        'magnification': magnification,
                        'type': sub_clazz,
                        'lesion': clazz
                    }, ignore_index=True)

dataset

,id,lesion,magnification,path,type
0,SOB_B_A_14-22549AB,benign,40X,../data/BreaKHis_v1/histology_slides/breast/be...,adenosis
1,SOB_B_A_14-22549AB,benign,40X,../data/BreaKHis_v1/histology_slides/breast/be...,adenosis
2,SOB_B_A_14-22549AB,benign,40X,../data/BreaKHis_v1/histology_slides/breast/be...,adenosis
3,SOB_B_A_14-22549AB,benign,40X,../data/BreaKHis_v1/histology_slides/breast/be...,adenosis
4,SOB_B_A_14-22549AB,benign,40X,../data/BreaKHis_v1/histology_slides/breast/be...,adenosis
...,...,...,...,...,...
7904,SOB_M_PC_14-19440,malignant,400X,../data/BreaKHis_v1/histology_slides/breast/ma...,papillary_carcinoma
7905,SOB_M_PC_14-19440,malignant,400X,../data/BreaKHis_v1/histology_slides/breast/ma...,papillary_carcinoma
7906,SOB_M_PC_14-19440,malignant,400X,../data/BreaKHis_v1/histology_slides/breast/ma...,papillary_carcinoma
7907,SOB_M_PC_14-19440,malignant,400X,../data/BreaKHis_v1/histology_slides/breast/ma...,papillary_carcinoma


In [6]:
x = []
for path in dataset["path"]:
    image = keras.preprocessing.image.load_img(path)
    x.append(keras.preprocessing.image.img_to_array(image))

y = [0 if clazz == "bening" else 1 for clazz in dataset["lesion"]]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

KeyboardInterrupt: 

In [ ]:
image_generator = preprocessing.ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2
    validation_split=0.2
    horizontal_flip=True,
    vertical_flip=True,
    shuffle=True
)

image_generator.fit(x_train)

In [ ]:
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(len(dataset),)),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(2, activation="sigmoid"),
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(
    datagen.flow(x_train, y_train, batch_size=32, subset='training'),
    validation_data=datagen.flow(x_train, y_train, batch_size=8, subset='validation'),
    steps_per_epoch=len(x_train) / 32,
    epochs=64
)